In [7]:
import sys
!{sys.executable} -m pip install requests

In [8]:
# GET THE CONFIG
import utils
user1 = utils.getenvcached("QUETZALCOATL_USER1")
token1 = utils.getenvcached("QUETZALCOATL_TOKEN1")

In [9]:
code = """
section .config
photon-number: 3
shots: 1000
max-shots: 10000

section .driver
qubo [[-1.0,0.2,2.0], \
      [0.2,-1.1,2.1], \
      [2.0, 2.1,-0.6]];
driver "SLOS";
// "SLOS";
// "qpu:ascella";
// "sim:slos";
// "sim:ascella";
filter-heuristic none;
optimize $all;
section .openoqasm
OPTICQASM 1.0;
generic_interferometer ps($phi) bs_rx($theta, $phi_tr) q[0], q[1], q[2];
"""

In [10]:
executor = "perceval"
nqubits = 1
config = {"naquada":1}
nshots = 1000

In [11]:
import requests
import json

url = "https://api2.anzaetek.com:443/execute"

def query24(executor, nqubits, config, nshots, circuit):
    q = {"__class__": "Assembly", 
        "nqubits": nqubits, 
        "executor": executor, 
        "config": config, 
        "circuit": circuit, 
        "nshots": nshots, 
        "user": user1, 
        "token": token1}
    query = {
        'user': user1, 
        'token': token1, 
        'query': json.dumps(q)}

    post_response = requests.post(url = url, json=query)
    rv = post_response.json()
    #print(rv)
    x = json.loads(rv)
    bestX = []
    bestE = 0.0
    for n in x["samples"]["naquada"]:
        #print(repr(n), n[1])
        for k in n[1]:
            #print(k)
            if k[1] < bestE:
                bestE = k[1]
                bestX = k[0]
    print("best: ", bestX, "energy @", bestE)
    return bestX, bestE

In [12]:
res = query24(executor, nqubits, config, nshots, code)
print(repr(res))

best:  [1, 1, 0] energy @ -1.7000000000000002
([1, 1, 0], -1.7000000000000002)
